In [2]:
import pandas as pd
import numpy as np
import datetime
import math
import quandl
pd.set_option('display.max_columns', 500)

## Read and Clean Data

In [10]:
def remove_foreign_reit_adr():
    '''
    @author: Wuding Li
    @The function returns a list of CUSIPs which not been filtered out
    @Reference: The function get the REIT and Foreign Stock filter through the Quandl database
    '''
    quandl.ApiConfig.api_key = "wJQ9TBnyt36w1a-dZjDg"
    CUSIPdic = quandl.get_table("SHARADAR/TICKERS", paginate=True)
    #get all domestic stock
    domestic = CUSIPdic[CUSIPdic['category'] == 'Domestic']
    #remove REIT
    keep = domestic[domestic['sicindustry'] != 'Real Estate Investment Trusts']
    #get CUSIP for all those keep ticker
    keep_list = keep['cusips'].values.tolist()
    #remove none in list
    res = [i for i in keep_list if i] 
    #seperate by space
    temp = " ".join(res).split(' ')
    #remove the last char in each elements to match CUSIP in dsf
    output = [x[:-1] for x in temp]
    return output

In [ ]:
#read in crsp dsf data
crsp = pd.read_csv('mydata_dsf.csv')
#apply the first filter
keep = remove_foreign_reit_adr()
crsp = crsp[crsp['CUSIP'].isin(keep)]
crsp['DATE'] =  pd.to_datetime(crsp['DATE'])

### Apply more filters and remove missing data

In [130]:
def check_closed_end_fund(data):
    '''
    @author: Wuding Li
    @The function return a list of PERMNO with no closed end fund
    @Variable: data: the input dataset
    '''
    
    date = data['DATE'].values
    s_date = [date[0],date[1],date[2]]
    PERMNO_list_start = (data[data['DATE'].isin(s_date)])['PERMNO'].values.tolist()
    
    e_date = [date[-1],date[-2],date[-3]]
    PERMNO_list_end = (data[data['DATE'].isin(e_date)])['PERMNO'].values.tolist()
    
    temp = set(PERMNO_list_end) 
    output = [value for value in PERMNO_list_start if value in temp] 
    
    return output

In [13]:
def check_more_than_5(data):
    '''
    @author: Wuding Li
    @The function returns a list of PERMNO which it's price in larger than $5 in the sampling and testing period
    @Variable: data: the input dataset
    '''
    mask = (data['PRC'] < 5)
    output = list(set((data.loc[mask])['PERMNO'].values.tolist()))
    return output

In [14]:
def remove_missing(data):
    '''
    @author: Wuding Li
    @The function returns a list of PERMMO that has missing values
    @Variable: data: the input dataset
    '''
    return list(set(data[data.isnull().any(axis=1)]['PERMNO'].values))

Above are all steps and functions for the raw dataset wrangling. The function{check_closed_end_fund}, function{check_more_than_5} and function{remove_missing} are called in the function{rolling_time_window} to save computational time and increase computation efficiency

## Generating samples 

In [104]:
def rolling_time_window(Start_Month, Start_Year, Time_Period_Training, Time_Period_Testing, Gap, data):
    '''
    @author: Wuding Li
    @The function returns two cleaned datasets. One is the sampling period data and the other is testing period data
    @variables: Start_Month: the starting month of our rolling time period
                Start_Year: the starting year of our rolling time period
                Time_Period_Training: Span of sampling Period
                Time_Period_Testing: Span of testing Period
                Gap: Gap between sampling and testing
                Data: The input data dataset
    @Also this function apply the filter no closed end fund and stock price more than 5 dollars
    '''
    
    #get the start and end date of data
    start_date_train = str(pd.date_range(start=str(Start_Month)+'/1/'+str(Start_Year), periods=1, freq='D')[0].date())
    end_date_train = str(pd.date_range(start=str(Start_Month)+'/1/'+str(Start_Year), periods=2, freq=str(Time_Period_Training-1)+'M')[-1].date())
    mask_train = (data['DATE'] >= start_date_train) & (data['DATE'] <= end_date_train)
    
    
    start_month_testing = Start_Month+Time_Period_Training+Gap
    start_year_testing = Start_Year + math.floor(start_month_testing/12)
    start_month_testing = start_month_testing % 12    
    start_date_test = str(pd.date_range(start=str(start_month_testing)+'/1/'+str(start_year_testing), periods=1, freq='D')[0].date())
    end_date_test = str(pd.date_range(start=str(start_month_testing)+'/1/'+str(start_year_testing), periods=2, freq=str(Time_Period_Testing-1)+'M')[-1].date())
    mask_test = (data['DATE'] >= start_date_test) & (data['DATE'] <= end_date_test)
    
    #applying filter functions below on the master dataset
    mask_master = (data['DATE'] >= start_date_train) & (data['DATE'] <= end_date_test)
    data_master = data.loc[mask_master]
    
    #check closed end fund
    keep_list = check_closed_end_fund(data_master) 
    #check more than 5 and missing data
    delete_list = check_more_than_5(data_master) + remove_missing(data_master)
    
    #comprehend both list
    keep_list = [x for x in keep_list if x not in delete_list]
    
    data_master = data_master[data_master['PERMNO'].isin(keep_list)]
    #remove strings in columns
    data_master[['RET']] = data_master[['RET']].apply(pd.to_numeric, errors='coerce').fillna(0).astype(float).dropna()
    data_master[['RETX']] = data_master[['RETX']].apply(pd.to_numeric, errors='coerce').fillna(0).astype(float).dropna()

    
    return data_master.loc[mask_train],data_master.loc[mask_test]

In [15]:
def get_monthly_ret_dsf(data):
    '''
    @author: Wuding Li
    @The function returns a dataset that convert daily return to monthly return
    @Variable: data: the input dataset
    '''
    data['ret'] = data['RET'].astype(float).apply(lambda x: x+1)
    data['YearMonth'] = data['DATE'].apply(lambda x:x.strftime('%Y-%m'))
    temp = data.groupby(['PERMNO','YearMonth']).agg({'ret': 'prod',
                                          'VOL': 'sum'}).reset_index()
    temp['ret'] = temp['ret'] - 1
    return temp

## Test Fama French Factors' Exposure 
### Run regression

In [121]:
def get_weight(data):
    '''
    @author: Wuding Li
    @The function add a column to the data set that assigned the weight to each stock for each month
    @Variable: data: the input dataset (Monthly data)                    
    '''
    count = len(set(data['PERMNO'].values))
    data['weight']=data.groupby(['YearMonth'])['ret'].apply(lambda x: (x-x.mean())*(-1/count))
    return data

### check missing values

In [274]:
#missing values check
def percent_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0,len(df_cols)):
        dict_x.update({df_cols[i]: round(data[df_cols[i]].isnull().mean()*100,2)})
    return dict_x
#features = train.iloc[:,-1:-6]
missing = percent_missing(train)
df_missing = sorted(missing.items(), key = lambda x: x[1], reverse = True)
print('percentage of missing data')
df_missing[0:10]

percentage of missing data


[('PERMNO', 0.0),
 ('DATE', 0.0),
 ('CUSIP', 0.0),
 ('PRC', 0.0),
 ('VOL', 0.0),
 ('ASKHI', 0.0),
 ('BIDLO', 0.0),
 ('RET', 0.0),
 ('RETX', 0.0),
 ('YEAR', 0.0)]

### create a sample to test